<a href="https://colab.research.google.com/github/dvschultz/stylegan3/blob/main/SG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN3

Just starting this...expect more updates soon.

In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# update to python 3.8
# someone can probably clean this up for me yeah?
!apt update
!apt upgrade
!apt dist-upgrade
!apt install build-essential zlib1g-dev libncurses5-dev libgdbm-dev libnss3-dev libssl-dev libreadline-dev libffi-dev wget
!apt install software-properties-common
!add-apt-repository ppa:deadsnakes/ppa -y
!apt update
!apt install python3.8
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3.8 get-pip.py
!python3.8 -m pip install --upgrade pip
!python3.8 -m pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html
!python3.8 -m pip install click scipy requests requests tqdm ninja matplotlib imageio imgui glfw pyopengl imageio-ffmpeg pyspng

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg3"):
    %cd "/content/drive/MyDrive/colab-sg3"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg3
    %cd colab-sg3
    !git clone https://github.com/dvschultz/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex click

## Convert Dataset
Pass a folder of images (just .pngs? TK) to create a zip file.

In [ ]:
!gdown --id 1XLbPLrOEiMy3aA86shCGPOW4V6fG7qDh -O /content/ladiescrop-1024.zip

In [ ]:
!unzip /content/ladiescrop-1024.zip -d /content/tmp

In [ ]:
!gdown --id 1qTR55ev8SPYTOOZGaJEvfROTWcjnLSRz -O /content/ladiescrop-sg2.pkl

In [ ]:
!python3.8 dataset_tool.py --source=/content/tmp/ladiescrop-1024 --dest=./datasets/ladiescrop-1024.zip

## Training


In [ ]:
!python train.py --outdir=./results --cfg=stylegan3-r --data=./datasets/ladiescrop-1024.zip \
--gpus=1 --batch=4 --gamma=10.0 --mirror=1 --kimg=5000 --snap=4 \
--resume=/content/ladiescrop-sg2.pkl